# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [17]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os
import math
from collections import OrderedDict

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle
from madminer.utils.particle import MadMinerParticle

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')


In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Observables and cuts

In [18]:
def resurrection_phi(leptons, photons, jets, met):
    # Parameters
    mw = 80.4
    
    # Particles
    if len(leptons) < 1 or len(photons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    a = photons[0]
    
    # Transverse mass and Delta
    mt = (mw**2 + (l+a).pt**2)**0.5
    deltasq = 0.
    if met.pt > 0. and l.pt > 0.:
        deltasq = (mw**2 - mt**2) / (2. * met.pt * l.pt)
        
    logging.debug('  Delta^2: %s', deltasq)
    
    # v reconstruction, "normal" case
    if deltasq > 0.:
        # Two solutions
        temp = np.log(1 + deltasq**0.5 * (2 + deltasq)**0.5 + deltasq)
        eta_v_plus = l.eta + temp
        eta_v_minus = l.eta - temp
        
        # Randomly select one of them
        dice = np.random.rand()
        if dice > 0.5:
            eta_v = eta_v_plus
        else:
            eta_v = eta_v_minus
            
    # v reconstruction, "other" case
    else:
        eta_v = l.eta
        
    logging.debug('  eta(v): %s', eta_v)
        
    # v particle
    v = MadMinerParticle()
    v.setptetaphim(met.pt, eta_v, met.phi(), 0.)
    
    # W and Wgamma reconstruction
    w = l + v
    vv = l + v + a
    
    # Boost into VV frame
    v_ = v.boost(vv.boostvector())
    l_ = l.boost(vv.boostvector())
    a_ = a.boost(vv.boostvector())
    w_ = w.boost(vv.boostvector())
    r_ = vv.boost(vv.boostvector())

    # Calculate axes of "special frame" (1708.07823)
    z_ = w_.vector / w_.p
    x_ = r_.vector - (r_.vector * z_) * z_
    x_ /= x_.mag
    y_ = z_.cross(x_)
    
    # Calculate x and y components of lepton wrt special x_, y_, z_ system
    lx_ = l.vector.dot(x_)
    ly_ = l.vector.dot(y_)
    
    # Calculate phi
    phi = math.atan2(ly_, lx_)
    
    return phi
    

In [19]:
# Number of particles considered
n_leptons = 1
n_photons = 1
n_jets = 1

def setup_observables(delphesprocessor):
    delphesprocessor.observables = OrderedDict()
    delphesprocessor.observables_required = OrderedDict()
    delphesprocessor.observables_defaults = OrderedDict()
        
    if False:
        # Default observables
        delphesprocessor.add_default_observables(
            n_leptons_max=n_leptons,
            n_photons_max=n_photons,
            n_jets_max=n_jets
        )

        # Correlations with MET
        for s1, i1 in [('l', 0), ('a', 0)]:
            delphesprocessor.add_observable(
                'deltaphi_{}{}_met'.format(s1, i1+1),
                '{}[{}].phi() - met.phi()'.format(s1, i1),
                required=True
            )

        # Reconstructed W
        delphesprocessor.add_observable(
            'm_l0_met',
            '(l[0] + met).m',
            required=True
        )
        delphesprocessor.add_observable(
            'pt_l0_met',
            '(l[0] + met).pt',
            required=True
        )

        # Selected correlations between particles
        for s1, i1, s2, i2 in [('l', 0, 'a', 0)]:
            delphesprocessor.add_observable(
                'm_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
                '({}[{}] + {}[{}]).m'.format(s1, i1, s2, i2),
                required=True
            )
            delphesprocessor.add_observable(
                'deltaeta_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
                '{}[{}].eta - {}[{}].eta'.format(s1, i1, s2, i2),
                required=True
            )
            delphesprocessor.add_observable(
                'deltaphi_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
                '{}[{}].phi() - {}[{}].phi()'.format(s1, i1, s2, i2),
                required=True
            )

        # Wgamma system
        delphesprocessor.add_observable(
            'm_a0_l0_met',
            '(a[0] + l[0] + met).m',
            required=True
        )
        delphesprocessor.add_observable(
            'pt_a0_l0_met',
            '(a[0] + l[0] + met).pt',
            required=True
        )
    
    # Resurection phi
    delphesprocessor.add_observable_from_function(
        'phi_resurrection',
        resurrection_phi,
        required=True
    )


In [13]:
def setup_cuts(delphesprocessor):
    delphesprocessor.add_cut(
        'n_ls > 0.1'
    )
    delphesprocessor.add_cut(
        'n_as > 0.1'
    )
    delphesprocessor.add_cut(
        'et_miss >= 10.'
    )

## Main loop

In [14]:
n_runs_per_benchmark = 18  # Number of run_cards

In [15]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=True)
    
    # Load events
    for benchmark in ['sm']:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        dp.add_hepmc_sample(
            mg_process_dir + 'Events/run_{}/tag_2_pythia8_events.hepmc.gz'.format(run_str),
            sampled_from_benchmark=benchmark
        )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_directory=log_dir,
        initial_command='source activate python2',
    )
    
    # Set up observables and cuts
    setup_observables(dp)
    setup_cuts(dp)
    
    # Analyse Delphes sample
    dp.analyse_delphes_samples(delete_delphes_files=True)
        
    # Save
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))


14:57  Starting analysis of runs for card 14
14:57  Benchmarks found in MadMiner file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
14:57  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz
14:57  Calling gunzip -k /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz > None
14:57  Found weight labels in HEPMC file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
14:57  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz
14:57  Unzipping /Users/johann

AttributeError: 'MadMinerParticle' object has no attribute 'boost_vector'

## Combine samples

In [14]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5')

19:08  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
19:08  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
19:08  Loading samples from file 1 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5
19:08  Loading samples from file 2 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_1.h5
19:08  Loading samples from file 3 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_2.h5
19:08  Loading samples from file 4 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/sam